Now to try to use our own GAT Pytorch implementation. Let's see how it goes.

In [140]:
import numpy as np
import torch_gat
import torch_geometric
import torch
import importlib
importlib.reload(torch_gat)

<module 'torch_gat' from '/home/ruy/Downloads/CS598-final/code/torch_gat.py'>

In [141]:
for dataset in ['citeseer', 'cora']:
    data = torch_geometric.datasets.Planetoid(root=f'../data/{dataset}', name=dataset)
    bias = torch.tensor(torch_gat.adj_to_bias(torch_gat.planetoid_adj_to_petarv_adj(data.edge_index).todense()[np.newaxis], [data.x.shape[0]], nhood=1), requires_grad=False)
    model = torch_gat.GAT(data.num_features, data.num_classes, bias)
    optimizer = torch.optim.Adam(model.parameters(), lr=.005, weight_decay=5e-4)

    for epoch in range(200):
        model.train()
        optimizer.zero_grad()
        out = model(data.x[np.newaxis]).squeeze()
        loss = torch.nn.functional.cross_entropy(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            print(out.argmax(dim=1)[data.train_mask].eq(data.y[data.train_mask]).sum().item() / int(data.train_mask.sum()))

    model.eval()
    out = model(data.x[np.newaxis]).squeeze()
    pred = out.argmax(dim=1)
    acc = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item() / int(data.test_mask.sum())
    print('\n\n*****************************************************************************************************\n')
    print(f'                                         {dataset} ')
    print(f'                                         Total Epochs: 200')
    print(f'                                         Test Accuracy: {acc:.4f}')
    print('\n*****************************************************************************************************\n\n')

0.21666666666666667
0.31666666666666665
0.4083333333333333
0.4583333333333333
0.4666666666666667
0.4
0.5083333333333333
0.475
0.4666666666666667
0.5083333333333333
0.49166666666666664
0.43333333333333335
0.5
0.49166666666666664
0.49166666666666664
0.4
0.5333333333333333
0.5916666666666667
0.475
0.49166666666666664


*****************************************************************************************************

                                         citeseer 
                                         Total Epochs: 200
                                         Test Accuracy: 0.2180

*****************************************************************************************************


0.12857142857142856
0.32857142857142857
0.3
0.39285714285714285
0.40714285714285714
0.45714285714285713
0.45714285714285713
0.34285714285714286
0.4642857142857143
0.4142857142857143
0.42857142857142855
0.42142857142857143
0.45714285714285713
0.37857142857142856
0.4714285714285714
0.4857142857142857
0